In [30]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import re
from scipy.sparse import hstack

In [2]:
for i in os.listdir():
    print(i)

task_1.ipynb
.ipynb_checkpoints
salary-train.csv
salary-test-mini.csv


In [3]:
df = pd.read_csv('salary-train.csv')

In [4]:
df_test = pd.read_csv('salary-test-mini.csv')

In [5]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [6]:
df_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [7]:
df.isnull().sum()

FullDescription           0
LocationNormalized        0
ContractTime          15582
SalaryNormalized          0
dtype: int64

In [8]:
df.ContractTime.fillna('nan', inplace=True)
df_test.ContractTime.fillna('nan', inplace=True)

In [9]:
df.columns

Index(['FullDescription', 'LocationNormalized', 'ContractTime',
       'SalaryNormalized'],
      dtype='object')

In [10]:
def clear_row(row):
    row = row.lower()
    row = re.sub('[^a-zA-Z0-9]', ' ', row)
    return row

In [11]:
df.FullDescription = df.FullDescription.map(clear_row)
df_test.FullDescription = df_test.FullDescription.map(clear_row)

In [12]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [13]:
vectorize = TfidfVectorizer(min_df=5)
X = vectorize.fit_transform(df.FullDescription)
X_test = vectorize.transform(df_test.FullDescription)

In [14]:
X

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [15]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [16]:
X_train_categ

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [18]:
X_full = hstack([X, X_train_categ])
X_full

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [19]:
X_full_test = hstack([X_test,X_test_categ])
X_full_test

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [21]:
y = df[df.columns[-1]]

In [22]:
y

0        33000
1        50000
2        40000
3        22500
4        20355
         ...  
59995    26400
59996    26000
59997    24500
59998    65000
59999    23040
Name: SalaryNormalized, Length: 60000, dtype: int64

In [27]:
model = Ridge(random_state=241)
model.fit(X_full, y)
predictions = model.predict(X_full)
predictions_test = model.predict(X_full_test)
print(mean_absolute_error(y, predictions))

6450.358192046452


In [28]:
predictions_test

array([56565.3254579 , 37140.63063337])

In [39]:
result = np.round(predictions_test, 2).astype(str)
result

array(['56565.33', '37140.63'], dtype='<U32')

In [40]:
with open('ans_1', 'w') as f:
    f.write(' '.join(result))